In [1]:
import numpy as np
import pandas as pd
import csv
from torch.utils.data import DataLoader
import polars as pl
from sklearn.cluster import MiniBatchKMeans
from tqdm import tqdm
from pathlib import Path

In [2]:
from skripsi_code.utils.domain_dataset import (
    MultiChunkParquet,
    MultiChunkDataset,
    EfficientChunkParquet,
)
from skripsi_code.utils.dataloader import random_split_dataloader

In [3]:
DATA_LIST = [
    "NF-BoT-IoT-v2",
    "NF-CSE-CIC-IDS2018-v2",
    "NF-ToN-IoT-v2",
    "NF-UNSW-NB15-v2",
]

In [4]:
DATA_PATH = "../data/parquet/"
train_data = MultiChunkParquet(
    DATA_PATH, DATA_LIST, domain=DATA_LIST, buffer_size=16, chunk_mode=True
)

# DATA_PATH = "../data/parquet/"
# train_data = EfficientChunkParquet(DATA_PATH, DATA_LIST, domain=DATA_LIST, chunk_mode=True)

# DATA_PATH = "../data/interim/"
# train_data = MultiChunkDataset(DATA_PATH, DATA_LIST, domain=DATA_LIST, buffer_size=512, chunk_mode=True)

Data directories: ['NF-BoT-IoT-v2', 'NF-CSE-CIC-IDS2018-v2', 'NF-ToN-IoT-v2', 'NF-UNSW-NB15-v2']


100%|██████████| 761/761 [00:00<00:00, 3024.55it/s]


In [7]:
len(train_data.buffer)

0

In [6]:
DATA_PATH = "../data/raw/NIDS_DATA/NF-CSE-CIC-IDS2018-v2.csv"
df_COMB = pl.read_csv(DATA_PATH).select(pl.nth(range(4, 42))).to_numpy()

In [7]:
df_COMB.shape[0]

18893708

In [7]:
# Initialize the MiniBatchKMeans model
n_clusters = 4  # Adjust this based on your needs
batch_size = 10000  # Set batch size for each model update
kmeans = MiniBatchKMeans(n_clusters=n_clusters, batch_size=batch_size)
kmeans.fit(df_COMB)

MiniBatchKMeans(batch_size=10000, n_clusters=4)

In [36]:
import glob

csv_files = list()
for directory in DATA_LIST:
    csv_files.extend(glob.glob(f"{DATA_PATH}/{directory}/*.csv"))

In [37]:
csv_files

['../data/interim//NF-BoT-IoT-v2/NF-BoT-IoT-v2_chunk_1.csv',
 '../data/interim//NF-BoT-IoT-v2/NF-BoT-IoT-v2_chunk_2.csv',
 '../data/interim//NF-BoT-IoT-v2/NF-BoT-IoT-v2_chunk_3.csv',
 '../data/interim//NF-BoT-IoT-v2/NF-BoT-IoT-v2_chunk_4.csv',
 '../data/interim//NF-BoT-IoT-v2/NF-BoT-IoT-v2_chunk_5.csv',
 '../data/interim//NF-BoT-IoT-v2/NF-BoT-IoT-v2_chunk_6.csv',
 '../data/interim//NF-BoT-IoT-v2/NF-BoT-IoT-v2_chunk_7.csv',
 '../data/interim//NF-BoT-IoT-v2/NF-BoT-IoT-v2_chunk_8.csv',
 '../data/interim//NF-BoT-IoT-v2/NF-BoT-IoT-v2_chunk_9.csv',
 '../data/interim//NF-BoT-IoT-v2/NF-BoT-IoT-v2_chunk_10.csv',
 '../data/interim//NF-BoT-IoT-v2/NF-BoT-IoT-v2_chunk_11.csv',
 '../data/interim//NF-BoT-IoT-v2/NF-BoT-IoT-v2_chunk_12.csv',
 '../data/interim//NF-BoT-IoT-v2/NF-BoT-IoT-v2_chunk_13.csv',
 '../data/interim//NF-BoT-IoT-v2/NF-BoT-IoT-v2_chunk_14.csv',
 '../data/interim//NF-BoT-IoT-v2/NF-BoT-IoT-v2_chunk_15.csv',
 '../data/interim//NF-BoT-IoT-v2/NF-BoT-IoT-v2_chunk_16.csv',
 '../data/interim

In [38]:
# Initialize the MiniBatchKMeans model
n_clusters = 4  # Adjust this based on your needs
batch_size = 10000  # Set batch size for each model update
kmeans = MiniBatchKMeans(n_clusters=n_clusters, batch_size=batch_size)

# Train on each file in chunks
for file in tqdm(csv_files):
    # Read the CSV file using Polars
    data = pl.read_csv(Path(file))
    features = data.select(pl.nth(range(4, 42))).to_numpy()
    kmeans.partial_fit(features)

# Optionally, save or use the trained model
# e.g., kmeans.cluster_centers_ or kmeans.labels_

100%|██████████| 1521/1521 [00:53<00:00, 28.58it/s]


In [44]:
# from sklearn.metrics import (adjusted_rand_score, normalized_mutual_info_score,
#                              fowlkes_mallows_score, silhouette_score,
#                              davies_bouldin_score, calinski_harabasz_score)
# from joblib import Parallel, delayed
# import polars as pl
# import numpy as np
#
# # Function to compute clustering metrics for a single CSV file
# def compute_metrics_for_file(file):
#     # Load the data from the CSV file
#     data = pl.read_csv(file)
#     data = data.select(pl.nth(range(4, 42))).to_numpy()
#
#     # Predict cluster labels
#     batch_labels = kmeans.predict(data)
#
#     # Initialize metrics dictionary
#     metrics = {'silhouette': None, 'davies_bouldin': None, 'calinski_harabasz': None}
#
#     # Compute metrics if there are at least two clusters
#     if len(np.unique(batch_labels)) > 1:
#         metrics['silhouette'] = silhouette_score(data, batch_labels)
#         metrics['davies_bouldin'] = davies_bouldin_score(data, batch_labels)
#         metrics['calinski_harabasz'] = calinski_harabasz_score(data, batch_labels)
#
#     return metrics, batch_labels
#
# # Initialize lists to collect labels for ARI, NMI, FMI if ground truth labels are available
# predicted_labels_all = []
# true_labels_all = []  # Only if you have ground-truth labels
# total_metrics = {'silhouette': 0, 'davies_bouldin': 0, 'calinski_harabasz': 0}
# total_samples = 0
#
# # Process each file in parallel
# results = Parallel(n_jobs=6)(delayed(compute_metrics_for_file)(Path(file)) for file in tqdm(csv_files, desc="Processing CSV files"))
#
# # Aggregate results from each file
# for result, batch_labels in results:
#     if result['silhouette'] is not None:
#         total_metrics['silhouette'] += result['silhouette'] * len(batch_labels)
#         total_metrics['davies_bouldin'] += result['davies_bouldin'] * len(batch_labels)
#         total_metrics['calinski_harabasz'] += result['calinski_harabasz'] * len(batch_labels)
#         total_samples += len(batch_labels)
#
#     # Collect labels for ARI, NMI, FMI if needed
#     predicted_labels_all.extend(batch_labels)
#     # Uncomment and extend with true labels if available
#     # true_labels_all.extend(...)
#
# # Compute final average scores for internal metrics
# average_silhouette_score = total_metrics['silhouette'] / total_samples
# average_davies_bouldin_score = total_metrics['davies_bouldin'] / total_samples
# average_calinski_harabasz_score = total_metrics['calinski_harabasz'] / total_samples
#
# # If you have true labels, calculate ARI, NMI, and FMI
# if true_labels_all:
#     ari = adjusted_rand_score(true_labels_all, predicted_labels_all)
#     nmi = normalized_mutual_info_score(true_labels_all, predicted_labels_all)
#     fmi = fowlkes_mallows_score(true_labels_all, predicted_labels_all)
#
#     print(f"Adjusted Rand Index (ARI): {ari:.3f}")
#     print(f"Normalized Mutual Information (NMI): {nmi:.3f}")
#     print(f"Fowlkes-Mallows Index (FMI): {fmi:.3f}")
#
# # Print internal metric scores
# print(f"Average Silhouette Score: {average_silhouette_score:.3f}")
# print(f"Average Davies-Bouldin Index: {average_davies_bouldin_score:.3f}")
# print(f"Average Calinski-Harabasz Index: {average_calinski_harabasz_score:.3f}")



Processing CSV files:   2%|▏         | 30/1521 [02:52<2:43:10,  6.57s/it]

KeyboardInterrupt: 

In [5]:
train_data

In [6]:
train_data.buffer[0].estimated_size("gb") * 100

1.0630395263433456

In [7]:
len(train_data)

761

In [8]:
import torch

train_data.buffer[0].select(pl.nth(range(4, 43))).cast(pl.Float32)

PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,MIN_TTL,MAX_TTL,LONGEST_FLOW_PKT,SHORTEST_FLOW_PKT,MIN_IP_PKT_LEN,MAX_IP_PKT_LEN,SRC_TO_DST_SECOND_BYTES,DST_TO_SRC_SECOND_BYTES,RETRANSMITTED_IN_BYTES,RETRANSMITTED_IN_PKTS,RETRANSMITTED_OUT_BYTES,RETRANSMITTED_OUT_PKTS,SRC_TO_DST_AVG_THROUGHPUT,DST_TO_SRC_AVG_THROUGHPUT,NUM_PKTS_UP_TO_128_BYTES,NUM_PKTS_128_TO_256_BYTES,NUM_PKTS_256_TO_512_BYTES,NUM_PKTS_512_TO_1024_BYTES,NUM_PKTS_1024_TO_1514_BYTES,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_ID,DNS_QUERY_TYPE,DNS_TTL_ANSWER,FTP_COMMAND_RET_CODE
f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
6.0,7.0,140.0,1.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,140.0,140.0,0.0,140.0,140.0,0.0,0.0,0.0,0.0,0.0,1.12e6,0.0,0.0,1.0,0.0,0.0,0.0,512.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6.0,7.0,280.0,2.0,0.0,0.0,2.0,2.0,0.0,4.294467e6,500.0,0.0,64.0,64.0,140.0,140.0,0.0,140.0,280.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,512.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6.0,7.0,280.0,2.0,0.0,0.0,2.0,2.0,0.0,4.29378e6,1187.0,0.0,64.0,64.0,140.0,140.0,0.0,140.0,140140.0,0.0,0.0,0.0,0.0,0.0,1.12e6,0.0,0.0,2.0,0.0,0.0,0.0,512.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17.0,188.0,56.0,2.0,0.0,0.0,0.0,0.0,0.0,4.294545e6,422.0,0.0,64.0,64.0,28.0,28.0,0.0,28.0,56.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17.0,188.0,56.0,2.0,0.0,0.0,0.0,0.0,0.0,4.293732e6,1235.0,0.0,64.0,64.0,28.0,28.0,0.0,28.0,2828.0,0.0,0.0,0.0,0.0,0.0,224000.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
17.0,188.0,84.0,3.0,0.0,0.0,0.0,0.0,0.0,4.29392e6,1047.0,0.0,64.0,64.0,28.0,28.0,0.0,28.0,5628.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17.0,188.0,56.0,2.0,0.0,0.0,0.0,0.0,0.0,4.29467e6,297.0,0.0,64.0,64.0,28.0,28.0,0.0,28.0,2828.0,0.0,0.0,0.0,0.0,0.0,224000.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6.0,7.0,140.0,1.0,40.0,1.0,22.0,2.0,20.0,4.294951e6,0.0,0.0,0.0,0.0,140.0,40.0,40.0,140.0,140.0,40.0,0.0,0.0,0.0,0.0,1.12e6,320000.0,1.0,1.0,0.0,0.0,0.0,512.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
train_data.reload_buffer()

In [10]:
train_dataloader = DataLoader(train_data, batch_size=1, shuffle=False, pin_memory=True)

In [11]:
train_features, train_labels = next(iter(train_dataloader))
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")

Feature batch shape: torch.Size([1, 100000, 39])
Labels batch shape: torch.Size([1, 100000])


In [12]:
train_features.reshape(-1, 39).size()

torch.Size([100000, 39])

In [13]:
train_features.reshape(-1, 39)

tensor([[  6.,   7., 140.,  ...,   0.,   0.,   0.],
        [  6.,   7., 280.,  ...,   0.,   0.,   0.],
        [  6.,   7., 280.,  ...,   0.,   0.,   0.],
        ...,
        [  6.,   7., 140.,  ...,   0.,   0.,   0.],
        [ 17., 188.,  56.,  ...,   0.,   0.,   0.],
        [ 17., 188.,  84.,  ...,   0.,   0.,   0.]])

In [14]:
len(train_dataloader)

761

In [15]:
if __name__ == "__main__":
    for X, Y in tqdm(train_dataloader, total=len(train_dataloader)):
        X = X.to("cuda")
        continue

100%|██████████| 761/761 [00:26<00:00, 28.88it/s]


In [16]:
train_features

tensor([[[  6.,   7., 140.,  ...,   0.,   0.,   0.],
         [  6.,   7., 280.,  ...,   0.,   0.,   0.],
         [  6.,   7., 280.,  ...,   0.,   0.,   0.],
         ...,
         [  6.,   7., 140.,  ...,   0.,   0.,   0.],
         [ 17., 188.,  56.,  ...,   0.,   0.,   0.],
         [ 17., 188.,  84.,  ...,   0.,   0.,   0.]]])

In [17]:
train_dataloader.dataset.reload_buffer()

In [18]:
source_domain = DATA_LIST[1:]
target_domain = DATA_LIST[0]
DATA_PATH = "../data/interim/"
DATA_PATH = "../data/parquet/"

train, val, test = random_split_dataloader(
    dir_path=DATA_PATH,
    source_dir=source_domain,
    target_dir=target_domain,
    source_domain=source_domain,
    target_domain=target_domain,
    batch_size=1,
    n_workers=0,
)

Data directories: ['NF-CSE-CIC-IDS2018-v2', 'NF-ToN-IoT-v2', 'NF-UNSW-NB15-v2']


100%|██████████| 383/383 [00:00<00:00, 2342.73it/s]


Data directories: ['NF-BoT-IoT-v2']


100%|██████████| 378/378 [00:00<00:00, 3345.38it/s]


Train: 364, Val: 19, Test: 378


In [19]:
train_features, train_labels = next(iter(train))
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")

Feature batch shape: torch.Size([1, 100000, 39])
Labels batch shape: torch.Size([1, 100000])


In [20]:
val_features, val_labels = next(iter(val))
print(f"Feature batch shape: {val_features.size()}")
print(f"Labels batch shape: {val_labels.size()}")

Feature batch shape: torch.Size([1, 100000, 39])
Labels batch shape: torch.Size([1, 100000])


In [21]:
test_features, test_labels = next(iter(test))
print(f"Feature batch shape: {test_features.size()}")
print(f"Labels batch shape: {test_labels.size()}")

Feature batch shape: torch.Size([1, 100000, 39])
Labels batch shape: torch.Size([1, 100000])


In [22]:
if __name__ == "__main__":
    for X, Y in tqdm(train, total=len(train)):
        X = X.to("cuda")
        continue
    for X, Y in tqdm(val, total=len(val)):
        X = X.to("cuda")
        continue
    for X, Y in tqdm(test, total=len(test)):
        X = X.to("cuda")
        continue

100%|██████████| 378/378 [00:13<00:00, 28.58it/s]
